<a href="https://colab.research.google.com/github/SvetlanaTsim/recommendation_systems/blob/main/lesson_02/hw_2_rec_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [265]:
!pip install implicit==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k


In [267]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [268]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# 1. Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [269]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
actual = result['actual']
#actual.head(2)

### Простой Random recommendation

In [270]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [271]:
sales_sum = data_train['sales_value'].sum()
sales_sum

7052176.859999996

In [272]:
uniquie_items_sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
uniquie_items_sales[:10]

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98
5,26426,2.29
6,26540,2.79
7,26601,7.59
8,26636,2.50
9,26691,8.37


In [273]:
uniquie_items_sales['sales_weights'] = uniquie_items_sales['sales_value'] / sales_sum
uniquie_items_sales[:10]

,item_id,sales_value,sales_weights
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07
2,26093,1.59,2.254623e-07
3,26190,1.54,2.183723e-07
4,26355,1.98,2.807644e-07
5,26426,2.29,3.247224e-07
6,26540,2.79,3.956225e-07
7,26601,7.59,1.076263e-06
8,26636,2.50,3.545005e-07
9,26691,8.37,1.186868e-06


In [274]:
np.round(uniquie_items_sales['sales_weights'].sum())

1.0

In [275]:
weights = uniquie_items_sales['sales_weights'].tolist()
weights[:15]

[2.969295923188179e-06,
 1.403821854802434e-07,
 2.2546229789251216e-07,
 2.183722885248231e-07,
 2.807643709604868e-07,
 3.2472242904015903e-07,
 3.9562252271704964e-07,
 1.0762634220151994e-06,
 3.5450046838445307e-07,
 1.1868675681511491e-06,
 3.899505152228984e-07,
 7.075829348953684e-07,
 1.4180018735378123e-07,
 1.020961348947225e-06,
 6.097408056212592e-07]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [276]:
#для того, чтобы веса складывались в единицу взяла не логарифм, а долю каждого товара в общей сумме продаж

def weighted_random_recommendation(items, items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False, p=items_weights)
    
    return recs.tolist()

In [277]:
result = pd.read_csv('predictions_basic.csv')
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [278]:
result['actual'] = actual

In [280]:
cols_str = ['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf', 'own_purchases']
for col in cols_str:
  result[col] = result[col].apply(lambda x: x.replace('[', '').replace(']', '').replace(',', '').replace('/n', '').split())
  result[col] = result[col].apply(lambda x: list(map(int, x)))

#result['itemitem'] = result['itemitem'].apply(lambda x: x.replace('[', '').replace(']', '').replace(',', '').split())

In [281]:
%%time
items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))
result.head(2)

CPU times: user 14.5 s, sys: 70.7 ms, total: 14.6 s
Wall time: 17.5 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[6979161, 10456176, 5582169, 5979256, 1276919]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[974626, 12604177, 9677423, 905501, 822367]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [282]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[6979161, 10456176, 5582169, 5979256, 1276919]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[974626, 12604177, 9677423, 905501, 822367]"


In [283]:
# your_code
metrics_df = pd.DataFrame(columns=['model', 'precision@3', 'precision@4', 'precision@5'])
metrics_df

,model,precision@3,precision@4,precision@5


In [284]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [285]:
result.columns.tolist()

['user_id',
 'actual',
 'random_recommendation',
 'popular_recommendation',
 'itemitem',
 'cosine',
 'tfidf',
 'own_purchases',
 'weighted_random_recommendation']

In [286]:
cols = ['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf', 'own_purchases', 'weighted_random_recommendation']

In [287]:
for col in cols:
  print(col)
  pr_3 = result.apply(lambda row: precision_at_k(row[col], row['actual'], k=3), axis=1).mean()
  pr_4 = result.apply(lambda row: precision_at_k(row[col], row['actual'], k=4), axis=1).mean()
  pr_5 = result.apply(lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean()

  
  metrics_df = metrics_df.append({
      'model': col,
      'precision@3': pr_3,
      'precision@4': pr_3,
      'precision@5': pr_5
    }, ignore_index=True)

random_recommendation
popular_recommendation
itemitem
cosine
tfidf
own_purchases
weighted_random_recommendation


In [288]:
metrics_df

,model,precision@3,precision@4,precision@5
0,random_recommendation,0.000653,0.000653,0.000588
1,popular_recommendation,0.137773,0.137773,0.155240
2,itemitem,0.033464,0.033464,0.033595
3,cosine,0.035260,0.035260,0.035260
4,tfidf,0.038198,0.038198,0.036141
5,own_purchases,0.198335,0.198335,0.179987
6,weighted_random_recommendation,0.000490,0.000490,0.000588


### Выводы

Наилучшей оказалась модель own_purchases с результатом 0.198335 по метрикам precision@3,	precision@4

weighted_random_recommendation на precision@5 показывает такой же результат, как и random_recommendation, на precision@3,	precision@4 результаты хуже.
